# More Reading Material

* [The Math of Machine Learning](https://gwthomas.github.io/docs/math4ml.pdf)

# Taking Symbolic Derivatives with Python

In [1]:
from sympy import *
from IPython.display import display
from sympy.printing.mathml import mathml
from IPython.display import display, Math, Latex

x, y, z = symbols('x y z')
init_printing(use_unicode=True)

In [2]:
def mprint(e):
    display(Math(latex(e)))

In [3]:
mprint(x**2)

<IPython.core.display.Math object>

In [4]:
mprint(diff(2*x**3))

<IPython.core.display.Math object>

In [5]:
expr = (x**3 + x**2 - x - 1)/(x**2 + 2*x + 1)
display(Math(latex(expr)))
expr = simplify(expr)
print(type(expr))
print(latex(expr))
display(Math(latex(expr)))

<IPython.core.display.Math object>

<class 'sympy.core.add.Add'>
x - 1


<IPython.core.display.Math object>

In [7]:
from IPython.display import display, Math, Latex
display(Math('\\frac{1_a^x}{2}'))

<IPython.core.display.Math object>

In [8]:
print(expr.subs(x,5))

4


In [9]:
eql = Eq(3*x+5,10)

In [10]:
z = solveset(eql,x)
display(Math(latex(z)))

<IPython.core.display.Math object>

In [11]:
from sympy import *
x, y, z = symbols('x y z')
init_printing(use_unicode=True)
expr = diff(sin(x)/x**2, x)
mprint(expr)

<IPython.core.display.Math object>

In [12]:
expr_i = integrate(expr,x)
mprint(expr_i)

<IPython.core.display.Math object>

# Keras Customization: Loss and Activation Functions

Your functions must be defined with TensorFlow graph commands.  The derivative will be taken automatically. (assuming all components of your function are differentiable)

# TensorFlow for Calculation

In [14]:
import tensorflow as tf

tf.multiply(tf.constant(2.0),tf.constant(5.0)).numpy()

10.0

In [15]:
import numpy as np
tf.multiply(np.array([2,4]),np.array([2,4]))

<tf.Tensor: id=10, shape=(2,), dtype=int64, numpy=array([ 4, 16])>

In [17]:
tf.multiply(2,4).numpy()

8

In [18]:
tf.divide(2,4)

In [20]:
tf.pow(2.0,4).numpy()

16.0

In [21]:
x = 5.0
y = tf.divide(1.0,tf.add(1,tf.exp(tf.negative(x))))
y

<tf.Tensor: id=34, shape=(), dtype=float32, numpy=0.9933072>

# Calculus with TensorFlow

How do we take derivatives?

* [Symbolic differentiation](http://tutorial.math.lamar.edu/pdf/common_derivatives_integrals.pdf)
* [Numerical differentiation](https://en.wikipedia.org/wiki/Finite_difference) (the method of finite differences)
* [Automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation)

Take the derivative of $f(x) = x^2$.

Symbolic derivative $f'(x) = rx^{r-1}$

$f(4) = 4^2 = 16$

$f'(4) = 2 \cdot 4 = 8$

This can be done in TensorFlow:

In [22]:
x = tf.constant(4.0)

with tf.GradientTape() as t:
  t.watch(x)
  z = tf.multiply(x, x)

# Derivative of z with respect to the original input tensor x
dz_dx = t.gradient(z, x)
print(dz_dx)

tf.Tensor(8.0, shape=(), dtype=float32)


Lets express the [Logistic function](https://en.wikipedia.org/wiki/Logistic_function) in TensorFlow. This is also called the Sigmoid Activation function in neural network literature.

$f(x) = \frac{1}{1 + e^{-x}}$

Written in TensorFlow:

In [23]:
x = tf.constant([5.0])
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.divide(1.0,tf.add(1,tf.exp(tf.negative(x))))
    
print(y)
dy_dx = t.gradient(y, x)
print(dy_dx)

tf.Tensor([0.9933072], shape=(1,), dtype=float32)
tf.Tensor([0.00664806], shape=(1,), dtype=float32)


Lets check the regular function.

In [24]:
import math

1/(1+math.exp(-5))

And lets check the derivative:
    
$f'(x) = \frac{e^x}{(e^x + 1)^2}$


In [25]:
math.exp(-5)/(math.exp(-5)+1)**2

In [ ]:
x = tf.ones((2, 2))
y = tf.reduce_sum(x)
z = tf.multiply(y, y)
y.numpy()

How to take second (and beyond) derivatives:

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  with tf.GradientTape() as gg:
    gg.watch(x)
    y = x * x
  dy_dx = gg.gradient(y, x)     # Will compute to 6.0
d2y_dx2 = g.gradient(dy_dx, x)  # Will compute to 2.0

# Custom Loss (Objective) Function

$ \operatorname{RMSE}=\sqrt{\frac{\sum_{t=1}^T (\hat y_t - y_t)^2}{T}} $

In [27]:
def mean_pred(y_true, y_pred):
    return tf.sqrt(tf.divide(tf.reduce_sum(tf.pow(tf.subtract(y_true, y_pred),2.0)),tf.cast(tf.size(y_true), tf.float32)))

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss=mean_pred, optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

W0930 19:48:43.874238 140735522108288 deprecation.py:323] From /Users/jheaton/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 398 samples
Epoch 1/100
398/398 - 0s - loss: 11.9127
Epoch 2/100
398/398 - 0s - loss: 9.8074
Epoch 3/100
398/398 - 0s - loss: 8.4966
Epoch 4/100
398/398 - 0s - loss: 7.8323
Epoch 5/100
398/398 - 0s - loss: 7.3702
Epoch 6/100
398/398 - 0s - loss: 7.6900
Epoch 7/100
398/398 - 0s - loss: 7.1135
Epoch 8/100
398/398 - 0s - loss: 6.9701
Epoch 9/100
398/398 - 0s - loss: 6.6870
Epoch 10/100
398/398 - 0s - loss: 7.2692
Epoch 11/100
398/398 - 0s - loss: 6.2498
Epoch 12/100
398/398 - 0s - loss: 7.7551
Epoch 13/100
398/398 - 0s - loss: 7.5446
Epoch 14/100
398/398 - 0s - loss: 7.5419
Epoch 15/100
398/398 - 0s - loss: 6.2173
Epoch 16/100
398/398 - 0s - loss: 5.4177
Epoch 17/100
398/398 - 0s - loss: 5.7693
Epoch 18/100
398/398 - 0s - loss: 5.4172
Epoch 19/100
398/398 - 0s - loss: 5.1264
Epoch 20/100
398/398 - 0s - loss: 5.1853
Epoch 21/100
398/398 - 0s - loss: 4.8951
Epoch 22/100
398/398 - 0s - loss: 4.8044
Epoch 23/100
398/398 - 0s - loss: 5.6221
Epoch 24/100
398/398 - 0s - loss: 5.1763
Epo

# Custom Activation (Transfer) Functions

In [29]:
import tensorflow as tf
def elliot_sym(x):
    return tf.divide(x,tf.add(1.0,tf.abs(x)))     

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Build the neural network
sgd = tf.keras.optimizers.SGD(lr=1e-10, decay=1e-6, momentum=0.9, nesterov=True)
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation=elliot_sym)) # Hidden 1
model.add(Dense(10, activation=elliot_sym)) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=400)

Train on 398 samples
Epoch 1/400
398/398 - 0s - loss: 580.7997
Epoch 2/400
398/398 - 0s - loss: 559.8733
Epoch 3/400
398/398 - 0s - loss: 542.7060
Epoch 4/400
398/398 - 0s - loss: 528.3312
Epoch 5/400
398/398 - 0s - loss: 517.3115
Epoch 6/400
398/398 - 0s - loss: 507.8317
Epoch 7/400
398/398 - 0s - loss: 497.6923
Epoch 8/400
398/398 - 0s - loss: 488.9701
Epoch 9/400
398/398 - 0s - loss: 481.3325
Epoch 10/400
398/398 - 0s - loss: 473.9950
Epoch 11/400
398/398 - 0s - loss: 466.9197
Epoch 12/400
398/398 - 0s - loss: 459.9756
Epoch 13/400
398/398 - 0s - loss: 453.2480
Epoch 14/400
398/398 - 0s - loss: 446.6219
Epoch 15/400
398/398 - 0s - loss: 440.1158
Epoch 16/400
398/398 - 0s - loss: 433.7691
Epoch 17/400
398/398 - 0s - loss: 427.5683
Epoch 18/400
398/398 - 0s - loss: 421.3823
Epoch 19/400
398/398 - 0s - loss: 415.3867
Epoch 20/400
398/398 - 0s - loss: 409.4597
Epoch 21/400
398/398 - 0s - loss: 403.6996
Epoch 22/400
398/398 - 0s - loss: 397.9255
Epoch 23/400
398/398 - 0s - loss: 392.3375

Epoch 191/400
398/398 - 0s - loss: 66.1705
Epoch 192/400
398/398 - 0s - loss: 65.9765
Epoch 193/400
398/398 - 0s - loss: 65.7886
Epoch 194/400
398/398 - 0s - loss: 65.5992
Epoch 195/400
398/398 - 0s - loss: 65.4229
Epoch 196/400
398/398 - 0s - loss: 65.2662
Epoch 197/400
398/398 - 0s - loss: 65.1019
Epoch 198/400
398/398 - 0s - loss: 64.9430
Epoch 199/400
398/398 - 0s - loss: 64.7934
Epoch 200/400
398/398 - 0s - loss: 64.6446
Epoch 201/400
398/398 - 0s - loss: 64.5025
Epoch 202/400
398/398 - 0s - loss: 64.3684
Epoch 203/400
398/398 - 0s - loss: 64.2242
Epoch 204/400
398/398 - 0s - loss: 64.1108
Epoch 205/400
398/398 - 0s - loss: 63.9893
Epoch 206/400
398/398 - 0s - loss: 63.8638
Epoch 207/400
398/398 - 0s - loss: 63.7647
Epoch 208/400
398/398 - 0s - loss: 63.6336
Epoch 209/400
398/398 - 0s - loss: 63.5306
Epoch 210/400
398/398 - 0s - loss: 63.4151
Epoch 211/400
398/398 - 0s - loss: 63.3242
Epoch 212/400
398/398 - 0s - loss: 63.2086
Epoch 213/400
398/398 - 0s - loss: 63.1222
Epoch 214/4

Epoch 382/400
398/398 - 0s - loss: 60.9409
Epoch 383/400
398/398 - 0s - loss: 60.9425
Epoch 384/400
398/398 - 0s - loss: 60.9407
Epoch 385/400
398/398 - 0s - loss: 60.9410
Epoch 386/400
398/398 - 0s - loss: 60.9468
Epoch 387/400
398/398 - 0s - loss: 60.9434
Epoch 388/400
398/398 - 0s - loss: 60.9440
Epoch 389/400
398/398 - 0s - loss: 60.9433
Epoch 390/400
398/398 - 0s - loss: 60.9418
Epoch 391/400
398/398 - 0s - loss: 60.9458
Epoch 392/400
398/398 - 0s - loss: 60.9443
Epoch 393/400
398/398 - 0s - loss: 60.9424
Epoch 394/400
398/398 - 0s - loss: 60.9415
Epoch 395/400
398/398 - 0s - loss: 60.9417
Epoch 396/400
398/398 - 0s - loss: 60.9427
Epoch 397/400
398/398 - 0s - loss: 60.9402
Epoch 398/400
398/398 - 0s - loss: 60.9490
Epoch 399/400
398/398 - 0s - loss: 60.9405
Epoch 400/400
398/398 - 0s - loss: 60.9441
